# Radial Metrics

In [ ]:
from pathlib import Path
from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")

import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

sample_file_ideal = Path("../../_static/sample_files/ideal_linear_pressure_increase.xplt")
lv_ideal = LV.from_file(sample_file_ideal) 
lv_ideal.identify_regions(geo_type=LV_GEO_TYPES.IDEAL,
                          apex_base_args=dict(ab_ql=0.05, ab_qh=0.97),
                          recompute_apex_base=dict(ql=0.05, qh=0.95)
                          )

import numpy as np
import logging

endo_long = []
epi_long = []
for i, a in enumerate(np.linspace(0, np.pi, 6, endpoint=False)):
    
    spk = lv_ideal.create_speckles(
        collection="long-6",
        group="endo",
        name=str(i),
        from_nodeset=LV_SURFS.ENDO,
        exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
        d=1.75,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        cluster_criteria="z2",
        n_clusters=10,
        t=0.0,
        kmin=-1,
        kmax=0.95,
        log_level=logging.WARN,
    )
    endo_long.append(spk)

    spk = lv_ideal.create_speckles(
        collection="long-6",
        group="epi",
        name=str(i),
        from_nodeset=LV_SURFS.EPI,
        exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
        d=2.4,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        cluster_criteria="z2",
        n_clusters=10,
        t=0.0,
        kmin=-1,
        kmax=0.95,
        log_level=logging.WARN,
    )
    epi_long.append(spk)

endo_circ = []
epi_circ = []

names = ["subapex", "apex", "superapex", "submid", "mid", "supermid", "subbase", "base", "superbase"]
for i, a in enumerate(np.linspace(0.25, 0.95, len(names), endpoint=False)):
    
    spk = lv_ideal.create_speckles(
        collection="circ-6",
        group="endo",
        name=names[i],
        from_nodeset=LV_SURFS.ENDO,
        d=1.75,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        cluster_criteria="angles2",
        n_clusters=8,
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.WARN,
    )
    endo_circ.append(spk)

    spk = lv_ideal.create_speckles(
        collection="circ-6",
        group="epi",
        name=names[i],
        from_nodeset=LV_SURFS.EPI,
        d=1.75,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        cluster_criteria="angles2",
        n_clusters=8,
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.WARN,
    )
    epi_circ.append(spk)



_ = lv_ideal.create_speckles(
        collection="SAMPLE",
        group="epi",
        name="SAMPLE",
        from_nodeset=LV_SURFS.EPI,
        d=4.0,
        k=0.8,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        cluster_criteria="angles2",
        n_clusters=8,
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.WARN,
    )

_ = lv_ideal.create_speckles(
        collection="SAMPLE",
        group="endo",
        name="SAMPLE",
        from_nodeset=LV_SURFS.ENDO,
        d=4.0,
        k=0.8,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles",
        cluster_criteria="angles2",
        n_clusters=8,
        t=0.0,
        kmin=-1.0,
        kmax=-1.0,
        log_level=logging.WARN,
    )

_ = lv_ideal.compute_base_apex_ref_over_timesteps()

## Radial metrics

In our implementation, 'radial metrics' are defined as any metric that relates a point with the left ventricle's longitudinal axis. In a clinical setting, these metrics are used to compute radial shortening and radial strain. However, it is often unclear how to proper establish its calculation as different reference points related to the longitudinal axis can be selected. 
For instance, we can use a 'center' point located at a specified 'height' of the longitudinal axis and compute the vector length from this reference to a point on the endocardium surface. On the other hand, we can compute the perpendicular distance between a point on the endocardium and the longitudinal axis. 
With this in mind, we provided two distinct metrics for radial measurements: 'Radial Distance' and 'Radial Length'. 

In addition, our implementation consider the usage of [Speckles](../basic_definitions/speckles.ipynb), which helps to approximate a clinical setting, facilitates the computation for certain metrics and helps to minimizing errors due to noise in geometry by averaging values based on local regions. See [docs](../basic_definitions/speckles.ipynb) for details.

### Radial Distance

**Definition**: Perpendicular distance from a given point to normal vector (defined from longitudinal line)

The 'Radial distance' is computed as the perpendicular distance (shortest euclidean distance) between a point (based on speckles) at a given region of the geometry and the longitudinal axis. Considering tha such axis is aligned with the Z axis, this metric can be thought as the '2D radius' from a top view (xy plane). Here is an example:

In [ ]:
sample_spk = lv_ideal.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]
timestep = 0.0

In [ ]:
rd = lv_ideal.plot_speckles_radial_distances(sample_spk, t=timestep)

In [ ]:
rds = lv_ideal.radial_distance(sample_spk, 
                       recompute=True, 
                       log_level=logging.WARN, 
                       approach="moving_vector",
                       t=timestep
                       )

In [ ]:
rd, rds

### Radial Length

**Definition**:

In [ ]:
sample_spk = lv_ideal.get_speckles(
    spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]
rd = lv_ideal.plot_speckles_radial_lengths(sample_spk, t=1.0)

## Thinner speckles, better results

In [ ]:
circ_spks = lv_ideal.get_speckles(spk_name="base", spk_collection="circ-6")
rd = lv_ideal.plot_speckles_radial_lengths(circ_spks[-1], t=1.0)

## 